I Have skipped the push notification but craeated a separate path
Code checks if Pushover credentials are available before attempting to send notifications. If not configured, it skips the external request and just prints locally. This allows the app to run without a Pushover account, while still supporting it if credentials are provided.


In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)
# Initialize OpenAI client (requires OPENAI_API_KEY in .env).
openai = OpenAI()

# Pushover setup: Read user and token from environment variables.
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1.messages.json"

# Print status of Pushover credentials for debugging.
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")


In [ ]:
# Safe push() function: Checks for credentials before sending.
def push(message):
    print(f"Push: {message}")
    if pushover_user and pushover_token:
        try:
            payload = {"user": pushover_user, "token": pushover_token, "message": message}
            resp = requests.post(pushover_url, data=payload, timeout=5)
            if resp.ok:
                print("Pushover: sent")
            else:
                print(f"Pushover: failed {resp.status_code} {resp.text}")
        except Exception as e:
            print(f"Pushover error: {e}")
    else:
        print("Pushover not configured; skipping external request.")


In [ ]:
# Function to record user details: Calls push() to notify, then returns a confirmation dict.
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

# Function to record unknown questions: Calls push() to notify, then returns a confirmation dict.
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

# JSON schema for record_user_details tool: Defines the tool's name, description, and parameters for the LLM.
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user",
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it",
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context",
            },
        },
        "required": ["email"],
        "additionalProperties": False,
    },
}

# JSON schema for record_unknown_question tool: Similar to above.
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered",
            },
        },
        "required": ["question"],
        "additionalProperties": False,
    },
}

# List of tools: Passed to the OpenAI API so the LLM can call these functions.
tools = [{"type": "function", "function": record_user_details_json},
         {"type": "function", "function": record_unknown_question_json}]


In [ ]:
# Second, more elegant version of handle_tool_calls: Uses globals() to dynamically call functions.
# Avoids the if statement, making it easier to add new tools.
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results


In [ ]:
# Load LinkedIn PDF and summary text: Extracts text from PDF and reads summary file.
# Assumes files are in "me/" directory.
reader = PdfReader("/Users/rohankajgaonkar/projects/agents/1_foundations/me/rohan_resume.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("/Users/rohankajgaonkar/projects/agents/1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

# Set the name (change this to your own name).
name = "Rohan"


In [ ]:
# System prompt: Instructs the LLM to act as the person, using provided context.
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, " \
                f"particularly questions related to {name}'s career, background, skills and experience. " \
                f"Your responsibility is to represent {name} for interactions on the website as faithfully as possible. " \
                f"You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. " \
                f"Be professional and engaging, as if talking to a potential client or future employer who came across the website. " \
                f"If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. " \
                f"If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
# Chat function: Handles the conversation loop with the LLM.
# Builds messages, calls OpenAI with tools, handles tool calls if needed, and returns the response.
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        # Call the LLM with tools enabled.
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call tools, execute them and continue.
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content



In [ ]:
# Launch Gradio interface: Creates a chat UI for the app.
gr.ChatInterface(chat).launch(pwa = True, share = True)